In [1]:
import gym
import Lorenz_envs
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import warnings

from IPython import display
from torch.distributions import MultivariateNormal
from torch.optim import Adam
from torch.nn import Linear, Module, MSELoss, ReLU, Sequential

warnings.filterwarnings("ignore")

/home/phu/Dropbox/general_backup/anaconda3/envs/gen_env/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/Tetris-v5
  logger.warn("Overriding environment {}".format(id))
/home/phu/Dropbox/general_backup/anaconda3/envs/gen_env/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/Tetris-ram-v5
  logger.warn("Overriding environment {}".format(id))
/home/phu/Dropbox/general_backup/anaconda3/envs/gen_env/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/Pitfall2-v5
  logger.warn("Overriding environment {}".format(id))
/home/phu/Dropbox/general_backup/anaconda3/envs/gen_env/lib/python3.10/site-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment ALE/Pitfall2-ram-v5
  logger.warn("Overriding environment {}".format(id))
/home/phu/Dropbox/general_backup/anaconda3/envs/gen_env/lib/python3.10/site-packages/gym

## Feed-Forward Network

In [2]:
class PolicyNetwork(Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.model = Sequential(
            Linear(input_dim, 64),
            ReLU(),
            Linear(64, 64),
            ReLU(),
            Linear(64, output_dim)
        )
    
    def forward(self, observation):
        if isinstance(observation, np.ndarray):
            observation = torch.tensor(observation, dtype=torch.float)
        return self.model(observation)

## LE  Reward Functions

In [3]:
class LEReward(RewardStrategy):
    def __init__(self, gamma, gae_lambda):
        super().__init__(gamma, gae_lambda)

        
    def compute_LE(self,st,st1):
        self.le = 1/len(st) * np.ln(st1/st)
        
    def compute_advantages(self, s, masks):
        batch_rtgs = []
        for episode_rewards in reversed(rewards):
            discounted_reward = 0
            for reward in reversed(episode_rewards):
                discounted_reward = reward + discounted_reward * self.gamma
                batch_rtgs.insert(0, discounted_reward)
        
        batch_rtgs = torch.tensor(batch_rtgs, dtype=torch.float)
        adv = batch_rtgs - values
        return batch_rtgs, adv
    

NameError: name 'RewardStrategy' is not defined

## PPO

In [4]:
class PPO:
    def __init__(self, env, policy_class, reward_class, objective_class, seed: int = 260):
        # Initialize hyperparameters
        self.timesteps_per_batch = 5000
        self.max_timesteps_per_episode = 200
        self.updates_per_iteration = 5
        self.lr = 3e-4
        self.gamma = 0.99
        self.gae_lambda = 0.95
        self.epsilon = 0.2
        self.critic_discount = 0.5
        self.seed = seed
        torch.manual_seed(self.seed)

        # Initialize other parameters
        self.render = True
        self.render_interval = 10

        # Set environment params
        self.env = env
        self.observation_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.shape[0]

        # Initialize networks
        self.actor = policy_class(self.observation_dim, self.action_dim)
        self.critic = policy_class(self.observation_dim, 1)

        # Initialize advantage and objective functions
        self.reward_strategy = reward_class(self.gamma, self.gae_lambda)
        self.objective = objective_class(self.epsilon)

        # Initialize optimizers
        self.actor_optimizer = Adam(self.actor.parameters(), lr=self.lr)
        self.critic_optimizer = Adam(self.critic.parameters(), lr=self.lr)

        # Initialize covariance matrix
        self.cov_matrix = torch.diag(torch.full(size=(self.action_dim,), fill_value=0.5))

    
    def train(self, max_timesteps):
        current_timesteps, current_iterations = 0, 0
        while current_timesteps < max_timesteps:
            # Run batch simulations (set of trajectories from running policy in the environment)
            batch_obs, batch_acts, batch_log_probs, batch_rewards, batch_lens, batch_masks = self.batch_simulation()
            print (batch_obs)
            # Increment timesteps and iterations
            current_timesteps += np.sum(batch_lens)
            current_iterations += 1
            self.logger["current_timesteps"] = current_timesteps
            self.logger["current_iterations"] = current_iterations

            # Calculate returns and advantages
            values, _ = self.evaluate(batch_obs, batch_acts)
            batch_returns, adv = self.reward_strategy.compute_advantages(batch_rewards, values.detach(), batch_masks)

            # Normalize advantages
            adv = (adv - adv.mean()) / (adv.std() + 1e-10)

            # Update the networks
            for _ in range(self.updates_per_iteration):
                V_phi, log_probs = self.evaluate(batch_obs, batch_acts)
                policy_ratios = torch.exp(log_probs - batch_log_probs)

                # Calculate actor and critic losses
                actor_loss  = self.objective.get_loss(policy_ratios, adv)
                critic_loss = MSELoss()(V_phi, batch_returns)
                
                # Calculate gradients and perform backward propagation for actor/critic networks
                self.actor_optimizer.zero_grad()
                actor_loss.backward(retain_graph=True)
                self.actor_optimizer.step()
                self.critic_optimizer.zero_grad()
                critic_loss.backward()
                self.critic_optimizer.step()

                self.logger["actor_losses"].append(actor_loss.detach())
                self.logger["total_losses"].append(actor_loss.detach() + self.critic_discount * critic_loss.detach())

            self._log_summary()

    def batch_simulation(self):
        batch_obs = []
        batch_acts = []
        batch_log_probs = []
        batch_rewards = []
        batch_lens = []
        batch_masks = []

        current_iteration = self.logger["current_iterations"]
        t = 0
        while t < self.timesteps_per_batch:
            episode_rewards = []
            observation = self.env.reset()
            is_done = False
            episode_t = 0

            # Run an episode
            for episode_t in range(self.max_timesteps_per_episode):
                if self.render and len(batch_lens) == 0 and (current_iteration % self.render_interval == 0):
                    self.env.render()
                
                # Calculate an action and make that action in the environment
                batch_obs.append(observation)
                action, log_prob = self.get_action(observation)
                observation, reward, is_done, _ = self.env.step(action)

                # Track rewards, actions, and log probs
                episode_rewards.append(reward)
                batch_acts.append(action)
                batch_log_probs.append(log_prob)
                batch_masks.append(0 if is_done else 1)

                t += 1
                if is_done:
                    break

            batch_lens.append(episode_t + 1)
            batch_rewards.append(episode_rewards)
        
        # Cast everything we need to a tensor
        batch_obs = torch.tensor(batch_obs, dtype=torch.float)
        batch_acts = torch.tensor(batch_acts, dtype=torch.float)
        batch_log_probs = torch.tensor(batch_log_probs, dtype=torch.float)
        batch_rewards = torch.tensor(batch_rewards, dtype=torch.float)
        batch_masks = torch.tensor(batch_masks, dtype=torch.float)

        self.logger["batch_rewards"] = batch_rewards
        self.logger["batch_lens"] = batch_lens

        return batch_obs, batch_acts, batch_log_probs, batch_rewards, batch_lens, batch_masks

    def evaluate(self, observations, actions):
        # Calculate the values of each observation
        values = self.critic(observations).squeeze()

        # Calculate the log probabilities of each action
        mean = self.actor(observations)
        distribution = MultivariateNormal(mean, self.cov_matrix)
        log_probs = distribution.log_prob(actions)

        return values, log_probs
    
    def get_action(self, observation):
        # Sample the distribution for an action
        mean = self.actor(observation)
        distribution = MultivariateNormal(mean, self.cov_matrix)
        action = distribution.sample()

        # Calculate the log probability for the sampled action
        log_prob = distribution.log_prob(action)

        return action.detach().numpy(), log_prob.detach()

    def get_results(self):
        return self.results

In [5]:
MAX_TIMESTEPS = 50

class TrainingResult:
    def __init__(self, label, reward_class, objective_class):
        self.label = label
        self.reward_class = reward_class
        self.objective_class = objective_class
        self.results = {}
    
    def set_results(self, results):
        self.results = results

training_results = [
    TrainingResult("RewardsToGo-NoClip", RewardsToGo, SurrogateObjectiveNoClip),
    TrainingResult("RewardsToGo-Clip", RewardsToGo, SurrogateObjectiveNoClip),
    TrainingResult("GeneralizedAdvantage-NoClip", GeneralizedAdvantage, SurrogateObjectiveNoClip),
    TrainingResult("GeneralizedAdvantage-Clip", GeneralizedAdvantage, SurrogateObjectiveClip),
]

def train_model(training_result, max_timesteps=MAX_TIMESTEPS):
    with gym.make("pendulum-v1") as pendulum_env:
        ppo_model = PPO(pendulum_env, PolicyNetwork, training_result.reward_class, training_result.objective_class)
        ppo_model.train(max_timesteps=max_timesteps)
        training_result.set_results(ppo_model.get_results())
        display_results(training_result.results, training_result.label)

NameError: name 'RewardsToGo' is not defined

In [6]:
train_model(training_results[0])

NameError: name 'train_model' is not defined